**Step 1: Install Necessary Libraries**



In [1]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

**Step 2: Load Pretrained Model and Tokenizer**


In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

**Step 3: Apply Parameter-Efficient Fine-Tuning (LoRA)**



In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor


**Step 4: Load and Preprocess the Dataset for Fine-Tuning**

In [6]:
chat_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

This function ensures the input data aligns with the model’s requirements, reducing inconsistencies during training.

In [7]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instruction = ""
    inputs       = examples["question"]
    outputs      = examples["answer"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = chat_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [8]:
from datasets import load_dataset

dataset = load_dataset("BoltMonkey/psychology-question-answer", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/74.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/197180 [00:00<?, ? examples/s]

Map:   0%|          | 0/197180 [00:00<?, ? examples/s]

In [9]:
dataset

Dataset({
    features: ['answer', 'question', 'text'],
    num_rows: 197180
})

In [10]:
import pprint
#Here are a few examples of what the data looks like
pprint.pprint(dataset[250])
pprint.pprint(dataset[260])
pprint.pprint(dataset[270])

{'answer': "Sympathy involves feeling sorry for someone's situation, while "
           "empathy involves stepping into another person's shoes and feeling "
           'what they feel.',
 'question': 'What is the difference between sympathy and empathy?',
 'text': '\n'
         '### Instruction:\n'
         '\n'
         '\n'
         '### Input:\n'
         'What is the difference between sympathy and empathy?\n'
         '\n'
         '### Response:\n'
         "Sympathy involves feeling sorry for someone's situation, while "
         "empathy involves stepping into another person's shoes and feeling "
         'what they feel.<|end_of_text|>'}
{'answer': 'Carl Rogers was a humanistic psychologist who believed in '
           'client-centered therapy, where the therapist provides support '
           'without judgement and helps clients find their own solutions to '
           'their problems.',
 'question': 'Who is Carl Rogers and what was his contribution to psychology?',
 'text': 

**Step 5: Configure the Training Parameters**

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"  # Ensure this is set before any imports

from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        disable_tqdm=True,  # This will disable tqdm progress bars (optional)
    ),
)



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/197180 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


**Step 6: Start Training**

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 197,180 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


{'loss': 2.4532, 'grad_norm': 0.966062068939209, 'learning_rate': 4e-05, 'epoch': 4.05720661324678e-05}
{'loss': 2.2703, 'grad_norm': 0.9085440635681152, 'learning_rate': 8e-05, 'epoch': 8.11441322649356e-05}
Unsloth: Will smartly offload gradients to save VRAM!
{'loss': 1.9212, 'grad_norm': 0.760171115398407, 'learning_rate': 0.00012, 'epoch': 0.00012171619839740339}
{'loss': 2.0107, 'grad_norm': 0.9793866872787476, 'learning_rate': 0.00016, 'epoch': 0.0001622882645298712}
{'loss': 1.7582, 'grad_norm': 1.0778189897537231, 'learning_rate': 0.0002, 'epoch': 0.00020286033066233897}
{'loss': 1.896, 'grad_norm': 2.4806275367736816, 'learning_rate': 0.00019636363636363636, 'epoch': 0.00024343239679480678}
{'loss': 1.6035, 'grad_norm': 1.4350855350494385, 'learning_rate': 0.00019272727272727274, 'epoch': 0.00028400446292727456}
{'loss': 1.4472, 'grad_norm': 1.9276973009109497, 'learning_rate': 0.0001890909090909091, 'epoch': 0.0003245765290597424}
{'loss': 1.2747, 'grad_norm': 17.88201141357

**Step 7: Perform Inference with the Fine-Tuned Model to Evaluate output**

In [13]:
FastLanguageModel.for_inference(model) # For faster Inference

inputs = tokenizer(
[
    chat_prompt.format(
        "", # instruction - leave this blank!
        "Who is the founder of the psychoanalytic theory?", # input
        "", # output - leave this blank!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>\n### Instruction:\n\n\n### Input:\nWho is the founder of the psychoanalytic theory?\n\n### Response:\nSigmund Freud.<|end_of_text|>']

In [14]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [
        chat_prompt.format(
            "",  # instruction
            "Who is the founder of the psychoanalytic theory? ",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_output = tokenizer.batch_decode(outputs)[0]  # Decode the output

# Extracting the response part
response = decoded_output.split("### Response:")[-1].strip()  # Get text after "### Response:"
response = response.split("<|end_of_text|>")[0].strip()  # Remove the end token if present

print(response)


Sigmund Freud is the founder of the psychoanalytic theory.


In [15]:

FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [
        chat_prompt.format(
            "",  # instruction
            "Who proposed the concept of self-efficacy?",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_output = tokenizer.batch_decode(outputs)[0]

response = decoded_output.split("### Response:")[-1].strip()
response = response.split("<|end_of_text|>")[0].strip()

print(response)

Albert Bandura.


In [16]:
FastLanguageModel.for_inference(model)

inputs = tokenizer(
    [
        chat_prompt.format(
            "",  # instruction
            "Who is known for their work on classical conditioning?",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_output = tokenizer.batch_decode(outputs)[0]

response = decoded_output.split("### Response:")[-1].strip()
response = response.split("<|end_of_text|>")[0].strip()

print(response)

Pavlov


**Step 7: Saving, loading finetuned models**

In [17]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # If you want to save online
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # If you want to save online

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [18]:
#Let's zip our model folder
import shutil
import os
folder_path = "/content/lora_model"
zip_file_path = "/content/lora_model.zip"

shutil.make_archive(zip_file_path.replace(".zip", ""), 'zip', folder_path)

'/content/lora_model.zip'

In [19]:
#Now if we want to use our model again we can just load it:

if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", #model folder
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)

In [20]:
chat_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [21]:
inputs = tokenizer(
    [
        chat_prompt.format(
            "",  # instruction
            "What is heuristics?",  # input
            "",  # output
        )
    ],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
decoded_output = tokenizer.batch_decode(outputs)[0]

response = decoded_output.split("### Response:")[-1].strip()
response = response.split("<|end_of_text|>")[0].strip()

print(response)

Heuristics are mental shortcuts that people use to make decisions and solve problems. They allow individuals to make quick, efficient judgments, but they can also lead to errors or biases.
